In [1]:
#Import modules
import pandas as pd
import numpy as np
import os
#from fitter import Fitter, get_common_distributions, get_distributions
#import sweetviz as sv
import matplotlib
matplotlib.use('TkAgg')  
import seaborn as sns
import matplotlib.pyplot as plt
import jaydebeapi
import matplotlib.cm as cm
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker

import joblib
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder


#more
%matplotlib inline
import optbinning
import tqdm

#from optbinning import Scorecard, BinningProcess
#from optbinning import OptimalBinning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from optbinning.scorecard import plot_auc_roc, plot_cap, plot_ks
from sklearn.feature_selection import VarianceThreshold

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Display options settings
import locale

# Set the locale to the user's default setting
locale.setlocale(locale.LC_ALL, '')

# Define a function to format floats/ints with thousands separators
def format_with_commas(x):
    if isinstance(x, float):
        return '{:,.4f}'.format(x)
    elif isinstance(x, int):
        return '{:,}'.format(x)
    return x
# Set the display format for floating-point numbers with thousands separators
pd.options.display.float_format = format_with_commas

pd.set_option('display.max_columns', None)

In [3]:
import sys

sys.path.append(r"C:\Users\erastus.kaiba\db_credentials")

from db_utils import run_query


In [4]:


base_df = run_query("""WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE 
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30'
    )
),
OutcomeWindow AS (
    SELECT
        s.SNAPSHOT_DATE,
        a.ARRANGEMENT_ID AS LOAN_ID,
        MAX(a.Days_In_Arrears) AS MAX_DPD_90D
    FROM SnapshotDates s
    JOIN STGKE.STG_AA_ARREARS a
        ON a.EXTRACTION_DATE > s.SNAPSHOT_DATE
        AND a.EXTRACTION_DATE <= DATEADD(DAY,90,s.SNAPSHOT_DATE)
    GROUP BY s.SNAPSHOT_DATE, a.ARRANGEMENT_ID
)
SELECT 
    s.SNAPSHOT_DATE,
    cm.CUSTOMER_NUMBER AS CUSTOMER_ID,
    am.ARRANGEMENT_ID AS LOAN_ID,
    am.ACCOUNT_NUMBER,
    am.CATEGORY,
    am.CATEGORY_DESC,
    am.CURRENCY,
    am.REPAYMENT_AMT AS EMI,
    am.INTEREST_RATE,
    CONVERT(DECIMAL(18,0), COALESCE(ex.Deal_balance,0)) AS EXPOSURE_ACTUAL,
    CASE 
        WHEN am.CURRENCY='KES'
        THEN CONVERT(DECIMAL(18,0), COALESCE(ex.Deal_balance,0))
        ELSE CONVERT(DECIMAL(18,0), COALESCE(ex.Deal_balance,0)) * am.EXCHANGE_RATE
    END AS EXPOSURE_KES,
    COALESCE(pdo.Days_In_Arrears,0) AS DPD,
    DATEDIFF(MONTH, am.VALUE_DATE, s.SNAPSHOT_DATE) AS MOB,
    CASE WHEN o.MAX_DPD_90D >= 20 THEN 1 ELSE 0 END AS TARGET_DEFAULT_90D
FROM SnapshotDates s
JOIN dbcba.KE_Accounts_Master am
    ON am.EXTRACTION_DATE = s.SNAPSHOT_DATE
JOIN dbcba.KE_Customer_Master cm
    ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    AND cm.EXTRACTION_DATE = s.SNAPSHOT_DATE
LEFT JOIN stgke.STG_ECB_TOTAL_EXPOSURE ex
    ON ex.RECID = am.ACCOUNT_NUMBER
    AND ex.LOAD_DATE = s.SNAPSHOT_DATE
LEFT JOIN STGKE.STG_AA_ARREARS pdo
    ON pdo.ARRANGEMENT_ID = am.ARRANGEMENT_ID
    AND pdo.EXTRACTION_DATE = s.SNAPSHOT_DATE
LEFT JOIN OutcomeWindow o
    ON o.LOAN_ID = am.ARRANGEMENT_ID
    AND o.SNAPSHOT_DATE = s.SNAPSHOT_DATE
WHERE am.PRODUCT_LINE='LENDING'
AND am.ARR_STATUS='CURRENT'
AND cm.BUSINESS_SEGMENT IN ('270','250')
AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
AND COALESCE(pdo.Days_In_Arrears,0)=0
AND cm.CUSTOMER_NUMBER IS NOT NULL;

""")
base_df.head()

,SNAPSHOT_DATE,CUSTOMER_ID,LOAN_ID,ACCOUNT_NUMBER,CATEGORY,CATEGORY_DESC,CURRENCY,EMI,INTEREST_RATE,EXPOSURE_ACTUAL,EXPOSURE_KES,DPD,MOB,TARGET_DEFAULT_90D
0,2025-06-30 00:00:00,710286,AA20249001MB,7102860063,3215,Mortgage Loan,KES,15561.820000,13.3500,0,0.0000,0,57,0
1,2025-07-31 00:00:00,710286,AA20249001MB,7102860063,3215,Mortgage Loan,KES,15561.860000,13.3500,1054047,"1,054,047.0000",0,58,0
2,2025-08-31 00:00:00,710286,AA20249001MB,7102860063,3215,Mortgage Loan,KES,15561.860000,13.3500,0,0.0000,0,59,0
3,2025-09-30 00:00:00,710286,AA20249001MB,7102860063,3215,Mortgage Loan,KES,15561.860000,13.3500,1046097,"1,046,097.0000",0,60,0
4,2025-10-31 00:00:00,710286,AA20249001MB,7102860063,3215,Mortgage Loan,KES,15561.890000,13.3500,1042999,"1,042,999.0000",0,61,0


In [5]:
dto_df = run_query(""" 
/* ==========================================
   SNAPSHOT + TRENDED DTO (SIMPLE VERSION)
   ========================================== */

WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30'
    )
),
FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
),
BaseData AS (
    SELECT
        s.SNAPSHOT_DATE,
        t.CUSTOMER_ID,
        t.BOOKING_DATE,
        CAST(t.AMOUNT_LCY * -1 AS DECIMAL(18,2)) AS AMOUNT_LCY_1
    FROM SnapshotDates s
    JOIN dbcba.KE_ACCOUNT_TRANSACTIONS t
        ON t.BOOKING_DATE <= s.SNAPSHOT_DATE
       AND t.BOOKING_DATE > DATEADD(MONTH,-12,s.SNAPSHOT_DATE)
    WHERE t.AMOUNT_LCY < 0
      AND t.REVERSAL_MARKER <> 'R'
        AND t.CUSTOMER_ID IS NOT NULL
      AND (t.DEBIT_CUSTOMER <> t.CREDIT_CUSTOMER OR t.CREDIT_CUSTOMER IS NULL)
      AND t.TRANSACTION_CODE NOT IN
          ('433','234','938','939','940','941','991','992')
        AND t.CUSTOMER_ID IN (SELECT CUSTOMER FROM FilteredCustomers)
),

/* ==========================================
   Month-level Aggregation
   ========================================== */

Monthly AS (

SELECT
    SNAPSHOT_DATE,
    CUSTOMER_ID,

    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 0 THEN AMOUNT_LCY_1 ELSE 0 END) AS M1,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 1 THEN AMOUNT_LCY_1 ELSE 0 END) AS M2,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 2 THEN AMOUNT_LCY_1 ELSE 0 END) AS M3,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 3 THEN AMOUNT_LCY_1 ELSE 0 END) AS M4,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 4 THEN AMOUNT_LCY_1 ELSE 0 END) AS M5,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 5 THEN AMOUNT_LCY_1 ELSE 0 END) AS M6,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 6 THEN AMOUNT_LCY_1 ELSE 0 END) AS M7,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 7 THEN AMOUNT_LCY_1 ELSE 0 END) AS M8,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 8 THEN AMOUNT_LCY_1 ELSE 0 END) AS M9,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 9 THEN AMOUNT_LCY_1 ELSE 0 END) AS M10,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 10 THEN AMOUNT_LCY_1 ELSE 0 END) AS M11,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 11 THEN AMOUNT_LCY_1 ELSE 0 END) AS M12,

    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 0 THEN 1 END) AS C1,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 1 THEN 1 END) AS C2,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 2 THEN 1 END) AS C3,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 3 THEN 1 END) AS C4,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 4 THEN 1 END) AS C5,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 5 THEN 1 END) AS C6,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 6 THEN 1 END) AS C7,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 7 THEN 1 END) AS C8,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 8 THEN 1 END) AS C9,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 9 THEN 1 END) AS C10,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 10 THEN 1 END) AS C11,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE) = 11 THEN 1 END) AS C12

FROM BaseData
GROUP BY SNAPSHOT_DATE, CUSTOMER_ID
)

/* ==========================================
   Final Output With Trends
   ========================================== */

SELECT
    SNAPSHOT_DATE,
    CUSTOMER_ID,

    /* Quarter Totals */
    (M1+M2+M3)  AS Q1_DTO,
    (M4+M5+M6)  AS Q2_DTO,
    (M7+M8+M9)  AS Q3_DTO,
    (M10+M11+M12) AS Q4_DTO,

    /* Month-on-Month */
    CASE WHEN M2=0 THEN NULL ELSE ROUND((M1-M2)/NULLIF(M2,0),2) END AS M1M2_Change,
    CASE WHEN M3=0 THEN NULL ELSE ROUND((M2-M3)/NULLIF(M3,0),2) END AS M2M3_Change,
    CASE WHEN M4=0 THEN NULL ELSE ROUND((M3-M4)/NULLIF(M4,0),2) END AS M3M4_Change,

    /* Quarter-on-Quarter */
    CASE WHEN (M4+M5+M6)=0 THEN NULL
         ELSE ROUND(((M1+M2+M3)-(M4+M5+M6))
              / NULLIF((M4+M5+M6),0),2)
    END AS Q1Q2_Change,

    CASE WHEN (M7+M8+M9)=0 THEN NULL
         ELSE ROUND(((M4+M5+M6)-(M7+M8+M9))
              / NULLIF((M7+M8+M9),0),2)
    END AS Q2Q3_Change

FROM Monthly
ORDER BY SNAPSHOT_DATE, CUSTOMER_ID;

""")
dto_df.head()

,SNAPSHOT_DATE,CUSTOMER_ID,Q1_DTO,Q2_DTO,Q3_DTO,Q4_DTO,M1M2_Change,M2M3_Change,M3M4_Change,Q1Q2_Change,Q2Q3_Change
0,2025-06-30 00:00:00,1000003,"123,999.1500","186,712.1000",0.0000,0.0000,-1.0000,0.0100,-0.1300,-0.3400,NaN
1,2025-06-30 00:00:00,1000009,"133,923.0500","1,186,038.7200",0.0000,0.0000,-0.6100,NaN,-1.0000,-0.8900,NaN
2,2025-06-30 00:00:00,1000011,"243,785.2500","51,902.3500",0.0000,0.0000,0.2100,-0.3500,0.9800,3.7000,NaN
3,2025-06-30 00:00:00,1000012,0.0000,"43,706.9500",0.0000,0.0000,NaN,NaN,NaN,-1.0000,NaN
4,2025-06-30 00:00:00,1000014,"75,923.4500","74,986.2000",0.0000,0.0000,0.2800,1.3200,-0.7700,0.0100,NaN


In [6]:
cto_df = run_query(""" 
    /* ==========================================
   SNAPSHOT + TRENDED CTO (MODEL ALIGNED)
   ========================================== */

WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30'
    )
),

FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
),
                                    
/* ==========================================
   Base Credit Transactions (12M Rolling)
   ========================================== */

BaseCredits AS (
    SELECT
        s.SNAPSHOT_DATE,
        t.CUSTOMER_ID,
        t.BOOKING_DATE,
        CAST(t.AMOUNT_LCY AS DECIMAL(18,2)) AS AMOUNT_LCY
    FROM SnapshotDates s
    JOIN dbcba.KE_ACCOUNT_TRANSACTIONS t
        ON t.BOOKING_DATE <= s.SNAPSHOT_DATE
       AND t.BOOKING_DATE > DATEADD(MONTH,-12,s.SNAPSHOT_DATE)

    WHERE t.AMOUNT_LCY > 0
      AND t.REVERSAL_MARKER <> 'R'
    AND t.CUSTOMER_ID IS NOT NULL
      AND (t.DEBIT_CUSTOMER <> t.CREDIT_CUSTOMER OR t.DEBIT_CUSTOMER IS NULL)
      AND t.SYSTEM_ID NOT IN ('AA')
      AND t.TRANSACTION_CODE NOT IN (
            '1001','944','945','1085','766','949','991','941',
            '433','1006','85','234','859','940','992',
            '947','939','946','948','938','967'
      )
                   and t.CUSTOMER_ID IN (SELECT CUSTOMER FROM FilteredCustomers)
),

/* ==========================================
   Month-Level Aggregation
   ========================================== */

Monthly AS (

SELECT
    SNAPSHOT_DATE,
    CUSTOMER_ID,

    /* Amount Buckets */
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=0  THEN AMOUNT_LCY ELSE 0 END) AS M1,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=1  THEN AMOUNT_LCY ELSE 0 END) AS M2,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=2  THEN AMOUNT_LCY ELSE 0 END) AS M3,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=3  THEN AMOUNT_LCY ELSE 0 END) AS M4,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=4  THEN AMOUNT_LCY ELSE 0 END) AS M5,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=5  THEN AMOUNT_LCY ELSE 0 END) AS M6,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=6  THEN AMOUNT_LCY ELSE 0 END) AS M7,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=7  THEN AMOUNT_LCY ELSE 0 END) AS M8,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=8  THEN AMOUNT_LCY ELSE 0 END) AS M9,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=9  THEN AMOUNT_LCY ELSE 0 END) AS M10,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=10 THEN AMOUNT_LCY ELSE 0 END) AS M11,
    SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=11 THEN AMOUNT_LCY ELSE 0 END) AS M12,

    /* Count Buckets */
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=0  THEN 1 END) AS C1,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=1  THEN 1 END) AS C2,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=2  THEN 1 END) AS C3,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=3  THEN 1 END) AS C4,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=4  THEN 1 END) AS C5,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=5  THEN 1 END) AS C6,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=6  THEN 1 END) AS C7,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=7  THEN 1 END) AS C8,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=8  THEN 1 END) AS C9,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=9  THEN 1 END) AS C10,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=10 THEN 1 END) AS C11,
    COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=11 THEN 1 END) AS C12

FROM BaseCredits
GROUP BY SNAPSHOT_DATE, CUSTOMER_ID
)

/* ==========================================
   Final Output With Trends
   ========================================== */

SELECT
    SNAPSHOT_DATE,
    CUSTOMER_ID,

    /* Quarter Totals */
    (M1+M2+M3)   AS Q1_CTO,
    (M4+M5+M6)   AS Q2_CTO,
    (M7+M8+M9)   AS Q3_CTO,
    (M10+M11+M12) AS Q4_CTO,

    (C1+C2+C3)   AS Q1_CTO_Count,
    (C4+C5+C6)   AS Q2_CTO_Count,
    (C7+C8+C9)   AS Q3_CTO_Count,
    (C10+C11+C12) AS Q4_CTO_Count,

    /* Month-to-Month Amount Trends */
    CASE WHEN M2=0 THEN NULL ELSE ROUND((M1-M2)/NULLIF(M2,0),2) END AS M1M2_CTO_Amt_Change,
    CASE WHEN M3=0 THEN NULL ELSE ROUND((M2-M3)/NULLIF(M3,0),2) END AS M2M3_CTO_Amt_Change,
    CASE WHEN M4=0 THEN NULL ELSE ROUND((M3-M4)/NULLIF(M4,0),2) END AS M3M4_CTO_Amt_Change,
    CASE WHEN M5=0 THEN NULL ELSE ROUND((M4-M5)/NULLIF(M5,0),2) END AS M4M5_CTO_Amt_Change,
    CASE WHEN M6=0 THEN NULL ELSE ROUND((M5-M6)/NULLIF(M6,0),2) END AS M5M6_CTO_Amt_Change,
    CASE WHEN M7=0 THEN NULL ELSE ROUND((M6-M7)/NULLIF(M7,0),2) END AS M6M7_CTO_Amt_Change,
    CASE WHEN M8=0 THEN NULL ELSE ROUND((M7-M8)/NULLIF(M8,0),2) END AS M7M8_CTO_Amt_Change,
    CASE WHEN M9=0 THEN NULL ELSE ROUND((M8-M9)/NULLIF(M9,0),2) END AS M8M9_CTO_Amt_Change,
    CASE WHEN M10=0 THEN NULL ELSE ROUND((M9-M10)/NULLIF(M10,0),2) END AS M9M10_CTO_Amt_Change,
    CASE WHEN M11=0 THEN NULL ELSE ROUND((M10-M11)/NULLIF(M11,0),2) END AS M10M11_CTO_Amt_Change,
    CASE WHEN M12=0 THEN NULL ELSE ROUND((M11-M12)/NULLIF(M12,0),2) END AS M11M12_CTO_Amt_Change,

    /* Quarter-to-Quarter Amount Trends */
    CASE WHEN (M4+M5+M6)=0 THEN NULL
         ELSE ROUND(((M1+M2+M3)-(M4+M5+M6)) / NULLIF((M4+M5+M6),0),2)
    END AS Q1Q2_CTO_Amt_Change,

    CASE WHEN (M7+M8+M9)=0 THEN NULL
         ELSE ROUND(((M4+M5+M6)-(M7+M8+M9)) / NULLIF((M7+M8+M9),0),2)
    END AS Q2Q3_CTO_Amt_Change,

    CASE WHEN (M10+M11+M12)=0 THEN NULL
         ELSE ROUND(((M7+M8+M9)-(M10+M11+M12)) / NULLIF((M10+M11+M12),0),2)
    END AS Q3Q4_CTO_Amt_Change

FROM Monthly
ORDER BY SNAPSHOT_DATE, CUSTOMER_ID;

                      """)
cto_df.head()

,SNAPSHOT_DATE,CUSTOMER_ID,Q1_CTO,Q2_CTO,Q3_CTO,Q4_CTO,Q1_CTO_Count,Q2_CTO_Count,Q3_CTO_Count,Q4_CTO_Count,M1M2_CTO_Amt_Change,M2M3_CTO_Amt_Change,M3M4_CTO_Amt_Change,M4M5_CTO_Amt_Change,M5M6_CTO_Amt_Change,M6M7_CTO_Amt_Change,M7M8_CTO_Amt_Change,M8M9_CTO_Amt_Change,M9M10_CTO_Amt_Change,M10M11_CTO_Amt_Change,M11M12_CTO_Amt_Change,Q1Q2_CTO_Amt_Change,Q2Q3_CTO_Amt_Change,Q3Q4_CTO_Amt_Change
0,2025-06-30 00:00:00,1000003,"124,000.0000","185,763.3800",0.0000,0.0000,2,4,0,0,-1.0000,0.0100,-0.1100,0.0600,0.2700,NaN,NaN,NaN,NaN,NaN,NaN,-0.3300,NaN,NaN
1,2025-06-30 00:00:00,1000009,"112,500.0000","77,000.0000",0.0000,0.0000,3,4,0,0,0.0000,0.0000,0.0000,0.0000,17.7500,NaN,NaN,NaN,NaN,NaN,NaN,0.4600,NaN,NaN
2,2025-06-30 00:00:00,1000011,"240,000.0000","82,000.0000",0.0000,0.0000,3,2,0,0,0.0000,0.0000,0.0000,NaN,-1.0000,NaN,NaN,NaN,NaN,NaN,NaN,1.9300,NaN,NaN
3,2025-06-30 00:00:00,1000012,0.0000,"44,000.0000",0.0000,0.0000,0,17,0,0,NaN,NaN,NaN,-1.0000,4.2900,NaN,NaN,NaN,NaN,NaN,NaN,-1.0000,NaN,NaN
4,2025-06-30 00:00:00,1000014,"78,981.0000","63,579.7400",0.0000,0.0000,6,5,0,0,0.4300,1.5400,-0.7400,3.2400,-0.1100,NaN,NaN,NaN,NaN,NaN,NaN,0.2400,NaN,NaN


In [7]:
FID_df = run_query(""" 
WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE 
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30')
                   )
,
FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
)

SELECT
    s.SNAPSHOT_DATE,
    a.customer_number AS CUSTOMER_ID,
    COUNT(DISTINCT a.ACCOUNT_NUMBER) AS NO_OF_ACCOUNTS_FID
FROM SnapshotDates s
JOIN STGKE.STG_AA_ARREARS a
    ON a.EXTRACTION_DATE <= s.SNAPSHOT_DATE
JOIN DBCBA.KE_ACCOUNTS_LIST b
    ON a.ACCOUNT_NUMBER = b.ACCOUNT_NUMBER
WHERE a.Days_In_Arrears BETWEEN 1 AND 30
AND DATEDIFF(DAY,b.ARR_START_DATE,a.ARREARS_DATE) BETWEEN 1 AND 30
and a.customer_number in (SELECT CUSTOMER FROM FilteredCustomers)
GROUP BY s.SNAPSHOT_DATE, a.customer_number;

""")
FID_df.head()

,SNAPSHOT_DATE,CUSTOMER_ID,NO_OF_ACCOUNTS_FID
0,2025-06-30 00:00:00,1000388,1
1,2025-06-30 00:00:00,1001490,2
2,2025-06-30 00:00:00,1001810,1
3,2025-06-30 00:00:00,1002009,1
4,2025-06-30 00:00:00,1002465,1


In [8]:
unpaid_df = run_query("""
-- ================================================
-- Full Snapshot-Aligned Unpaid Feature Script
-- ================================================

WITH SnapshotDates AS (
    -- Step 1: Define your snapshot dates (as per model requirements)
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30'
    )
),
FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
),

BaseUnpaid AS (
    -- Step 2: Pull all unpaid transactions in the last 12 months relative to snapshot
    SELECT
        s.SNAPSHOT_DATE,
        t.CUSTOMER_ID,
        t.BOOKING_DATE,
        CAST(t.AMOUNT_LCY * -1 AS DECIMAL(18,2)) AS UNPAID_AMOUNT
    FROM SnapshotDates s
    JOIN dbcba.KE_ACCOUNT_TRANSACTIONS t
        ON t.BOOKING_DATE <= s.SNAPSHOT_DATE
       AND t.BOOKING_DATE > DATEADD(MONTH,-12,s.SNAPSHOT_DATE)
    WHERE t.AMOUNT_LCY < 0
      AND (t.DEBIT_CUSTOMER <> t.CREDIT_CUSTOMER OR t.CREDIT_CUSTOMER IS NULL)
      AND t.TRANSACTION_CODE IN (
          '938','939','940','941','944','945','946','947','948','949',
          '991','992','1084','1085','285','561','882','904','905','923'
      )
      AND t.REVERSAL_MARKER <> 'R'
                      and t.CUSTOMER_ID IN (SELECT CUSTOMER FROM FilteredCustomers)
),

Monthly AS (
    -- Step 3: Bucket transactions into 12 months relative to snapshot
    SELECT
        SNAPSHOT_DATE,
        CUSTOMER_ID,

        -- Amounts per month
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=0 THEN UNPAID_AMOUNT ELSE 0 END) AS M1,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=1 THEN UNPAID_AMOUNT ELSE 0 END) AS M2,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=2 THEN UNPAID_AMOUNT ELSE 0 END) AS M3,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=3 THEN UNPAID_AMOUNT ELSE 0 END) AS M4,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=4 THEN UNPAID_AMOUNT ELSE 0 END) AS M5,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=5 THEN UNPAID_AMOUNT ELSE 0 END) AS M6,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=6 THEN UNPAID_AMOUNT ELSE 0 END) AS M7,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=7 THEN UNPAID_AMOUNT ELSE 0 END) AS M8,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=8 THEN UNPAID_AMOUNT ELSE 0 END) AS M9,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=9 THEN UNPAID_AMOUNT ELSE 0 END) AS M10,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=10 THEN UNPAID_AMOUNT ELSE 0 END) AS M11,
        SUM(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=11 THEN UNPAID_AMOUNT ELSE 0 END) AS M12,

        -- Counts per month
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=0 THEN 1 END) AS C1,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=1 THEN 1 END) AS C2,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=2 THEN 1 END) AS C3,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=3 THEN 1 END) AS C4,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=4 THEN 1 END) AS C5,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=5 THEN 1 END) AS C6,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=6 THEN 1 END) AS C7,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=7 THEN 1 END) AS C8,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=8 THEN 1 END) AS C9,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=9 THEN 1 END) AS C10,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=10 THEN 1 END) AS C11,
        COUNT(CASE WHEN DATEDIFF(MONTH, BOOKING_DATE, SNAPSHOT_DATE)=11 THEN 1 END) AS C12

    FROM BaseUnpaid
    GROUP BY SNAPSHOT_DATE, CUSTOMER_ID
),

FinalUnpaid AS (
    -- Step 4: Calculate quarterly aggregates and trends
    SELECT
        SNAPSHOT_DATE,
        CUSTOMER_ID,

        -- Quarter values
        (M1+M2+M3)   AS Q1_Unpaid_Value,
        (M4+M5+M6)   AS Q2_Unpaid_Value,
        (M7+M8+M9)   AS Q3_Unpaid_Value,
        (M10+M11+M12) AS Q4_Unpaid_Value,

        (C1+C2+C3)   AS Q1_Unpaid_Counts,
        (C4+C5+C6)   AS Q2_Unpaid_Counts,
        (C7+C8+C9)   AS Q3_Unpaid_Counts,
        (C10+C11+C12) AS Q4_Unpaid_Counts,

        -- Quarter amount trends
        CASE WHEN (M4+M5+M6)=0 THEN NULL
             ELSE ROUND(((M1+M2+M3)-(M4+M5+M6))/NULLIF((M4+M5+M6),0),2)
        END AS Q1Q2_Unpaid_Amt_change_pct,

        CASE WHEN (M7+M8+M9)=0 THEN NULL
             ELSE ROUND(((M4+M5+M6)-(M7+M8+M9))/NULLIF((M7+M8+M9),0),2)
        END AS Q2Q3_Unpaid_Amt_change_pct,

        CASE WHEN (M10+M11+M12)=0 THEN NULL
             ELSE ROUND(((M7+M8+M9)-(M10+M11+M12))/NULLIF((M10+M11+M12),0),2)
        END AS Q3Q4_Unpaid_Amt_change_pct,

        -- Quarter count trends
        CASE WHEN (C4+C5+C6)=0 THEN NULL
             ELSE ROUND(((C1+C2+C3)-(C4+C5+C6))/NULLIF((C4+C5+C6),0),2)
        END AS Q1Q2_Unpaid_Cnt_Change_pct,

        CASE WHEN (C7+C8+C9)=0 THEN NULL
             ELSE ROUND(((C4+C5+C6)-(C7+C8+C9))/NULLIF((C7+C8+C9),0),2)
        END AS Q2Q3_Unpaid_Cnt_Change_pct,

        CASE WHEN (C10+C11+C12)=0 THEN NULL
             ELSE ROUND(((C7+C8+C9)-(C10+C11+C12))/NULLIF((C10+C11+C12),0),2)
        END AS Q3Q4_Unpaid_Cnt_Change_pct,

        -- Current vs previous month
        CASE WHEN M2=0 THEN NULL
             ELSE ROUND((M1-M2)/NULLIF(M2,0),2)
        END AS CurrMonthPrevMonth_Unpaid_Amt_change_pct,

        CASE WHEN C2=0 THEN NULL
             ELSE ROUND((C1-C2)/NULLIF(C2,0),2)
        END AS CurrMonthPrevMonth_Unpaid_Cnt_change_pct

    FROM Monthly
)

-- Final output
SELECT *
FROM FinalUnpaid
where CUSTOMER_ID IS NOT NULL
ORDER BY SNAPSHOT_DATE, CUSTOMER_ID;

""")

unpaid_df.head()


,SNAPSHOT_DATE,CUSTOMER_ID,Q1_Unpaid_Value,Q2_Unpaid_Value,Q3_Unpaid_Value,Q4_Unpaid_Value,Q1_Unpaid_Counts,Q2_Unpaid_Counts,Q3_Unpaid_Counts,Q4_Unpaid_Counts,Q1Q2_Unpaid_Amt_change_pct,Q2Q3_Unpaid_Amt_change_pct,Q3Q4_Unpaid_Amt_change_pct,Q1Q2_Unpaid_Cnt_Change_pct,Q2Q3_Unpaid_Cnt_Change_pct,Q3Q4_Unpaid_Cnt_Change_pct,CurrMonthPrevMonth_Unpaid_Amt_change_pct,CurrMonthPrevMonth_Unpaid_Cnt_change_pct
0,2025-06-30 00:00:00,1000272,0.0000,"219,765.0000",0.0000,0.0000,0,1,0,0,-1.0000,NaN,NaN,-1.0000,NaN,NaN,NaN,NaN
1,2025-06-30 00:00:00,1000769,"30,229.0000",0.0000,0.0000,0.0000,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-06-30 00:00:00,1001558,0.0000,"340,000.0000",0.0000,0.0000,0,1,0,0,-1.0000,NaN,NaN,-1.0000,NaN,NaN,NaN,NaN
3,2025-06-30 00:00:00,1001752,0.0000,"325,000.0000",0.0000,0.0000,0,1,0,0,-1.0000,NaN,NaN,-1.0000,NaN,NaN,NaN,NaN
4,2025-06-30 00:00:00,1003502,"50,447.0000",0.0000,0.0000,0.0000,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
maxdpd_df = run_query(""" 
WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE 
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30')),
                      
FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
)                   
SELECT
    s.SNAPSHOT_DATE,
    a.CUSTOMER AS CUSTOMER_ID,
    MAX(a.OD_DAYS) AS MAX_LOAN_DPD_60D
FROM SnapshotDates s
JOIN STGKE.STG_AA_BILL_ARREARS a
    ON a.EXTRACTION_DATE > DATEADD(DAY,-60,s.SNAPSHOT_DATE)
    AND a.EXTRACTION_DATE <= s.SNAPSHOT_DATE
                      and  a.CUSTOMER in (SELECT CUSTOMER FROM FilteredCustomers)
GROUP BY s.SNAPSHOT_DATE, a.CUSTOMER;
""")
maxdpd_df.head()

,SNAPSHOT_DATE,CUSTOMER_ID,MAX_LOAN_DPD_60D
0,2025-06-30 00:00:00,1000388,0
1,2025-06-30 00:00:00,1000451,0
2,2025-06-30 00:00:00,1000626,0
3,2025-06-30 00:00:00,1000775,0
4,2025-06-30 00:00:00,1001198,0


In [10]:
crb_df = run_query(""" 
                            
WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30'
    )
),
FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
),
LatestCRB AS (
    SELECT
        s.SNAPSHOT_DATE,
        c.CLIENTID AS CUSTOMER_ID,  -- keep as string
        c.NUMBEROF_ACCOUNTS,
        c.OPENACCOUNT,
        c.PERFORMING,
        c.NONPERFORMING,
        c.CLOSED,
        c.CURRENTINARREARS,
        c.ARREARS0DAYS,
        c.MAXARREARS,
        c.MAXARREARSLAST12MONTHS,
        c.MAXARREARSLAST3MONTHS,
        c.PRINCIPALAMOUNT,
        c.SCHEDULEDPAYMENTAMT,
        c.CREDITAPPLICATIONSDONE,
        c.CURRENTBALANCEAMOUNT,
        c.SUBSCRIBERCURRENTBALANCEAMOUNT,
        c.PASTDUEAMOUNT,
        c.NUMBEROFENQUIRIES,
        c.ENQUIRIES30DAYS,
        c.ENQUIRIES90DAYS,
        c.SCORE AS CRB_TU_SCORE,
        c.MOBILE_TOTAL,
        c.MOBILE_PERFORMING,
        c.MOBILE_NONPERFORMING,
        c.MOBILE_PRINCIPALAMOUNT,
        c.LOAD_DATE
    FROM SnapshotDates s
    JOIN dbcba.LEND_FOR_ALL_CRB_DATA c
      ON c.LOAD_DATE <= s.SNAPSHOT_DATE
    WHERE ISNUMERIC(c.CLIENTID) = 1  -- filter numeric only
                   and c.CLIENTID IN (SELECT CUSTOMER FROM FilteredCustomers)
),

CRB_Max AS (
    SELECT l.*
    FROM LatestCRB l
    JOIN (
        SELECT 
            SNAPSHOT_DATE,
            CUSTOMER_ID,
            MAX(LOAD_DATE) AS MAX_LOAD_DATE
        FROM LatestCRB
        GROUP BY SNAPSHOT_DATE, CUSTOMER_ID
    ) m
      ON l.SNAPSHOT_DATE = m.SNAPSHOT_DATE
     AND l.CUSTOMER_ID = m.CUSTOMER_ID
     AND l.LOAD_DATE = m.MAX_LOAD_DATE
)

SELECT *
FROM CRB_Max
ORDER BY SNAPSHOT_DATE, CUSTOMER_ID;


""")
crb_df.head()

,SNAPSHOT_DATE,CUSTOMER_ID,NUMBEROF_ACCOUNTS,OPENACCOUNT,PERFORMING,NONPERFORMING,CLOSED,CURRENTINARREARS,ARREARS0DAYS,MAXARREARS,MAXARREARSLAST12MONTHS,MAXARREARSLAST3MONTHS,PRINCIPALAMOUNT,SCHEDULEDPAYMENTAMT,CREDITAPPLICATIONSDONE,CURRENTBALANCEAMOUNT,SUBSCRIBERCURRENTBALANCEAMOUNT,PASTDUEAMOUNT,NUMBEROFENQUIRIES,ENQUIRIES30DAYS,ENQUIRIES90DAYS,CRB_TU_SCORE,MOBILE_TOTAL,MOBILE_PERFORMING,MOBILE_NONPERFORMING,MOBILE_PRINCIPALAMOUNT,LOAD_DATE
0,2025-06-30 00:00:00,1000003,55,2,55,0,53,0,55,0,0,0,321307.27,324303.49,0,27378.5,0,0,4,0,1,739,48,48,0,318186,2025-06-25
1,2025-06-30 00:00:00,1000009,32,0,32,0,32,0,32,0,0,0,132319.37,133106.87,0,0,0,0,7,0,0,677,1,1,0,10500,2025-06-25
2,2025-06-30 00:00:00,1000011,95,1,95,0,94,0,95,0,0,0,1341751.98,1445082.25,0,25590.33,0,0,9,0,0,710,82,82,0,1336000,2025-06-25
3,2025-06-30 00:00:00,1000012,22,1,21,1,20,1,21,534,534,0,17601.4,15858.24,0,9863.56,863.56,9000,19,0,2,566,3,2,1,7100,2025-06-25
4,2025-06-30 00:00:00,1000014,40,2,38,2,37,2,38,1719,1719,1719,164009.15,139209.55,0,45556.33,2752.09,42804.24,34,0,0,524,5,4,1,49700,2025-06-25


In [ ]:

#Pull Filtered Customers

filtered_customers_df = run_query("""
SELECT DISTINCT am.CUSTOMER AS CUSTOMER_ID
FROM dbcba.KE_Accounts_List am
INNER JOIN dbcba.KE_Customer_Master cm
    ON cm.CUSTOMER_NUMBER = am.CUSTOMER
WHERE am.PRODUCT_LINE='LENDING'
  AND am.ARR_STATUS='CURRENT'
  AND cm.BUSINESS_SEGMENT IN ('270','250')
  AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
  AND am.CUSTOMER IS NOT NULL
""")


In [ ]:
base_bal_df = run_query("""
SELECT
    acc.EXTRACTION_DATE AS SNAPSHOT_DATE,
    acc.CUSTOMER AS CUSTOMER_ID,
    avg_bal.Av_Bal_Date,
    avg_bal.CUMULATIVE_BAL_LCY,
    avg_bal.NUM_DAYS
FROM dbcba.KE_Accounts_master acc
INNER JOIN (
    SELECT DISTINCT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    INNER JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
) fc
    ON acc.CUSTOMER = fc.CUSTOMER
LEFT JOIN STGKE.STG_AVERAGE_BALANCE avg_bal
    ON avg_bal.Account_number = acc.Account_number
    AND avg_bal.Av_Bal_Date <= acc.EXTRACTION_DATE
    AND avg_bal.Av_Bal_Date > DATEADD(MONTH,-12,acc.EXTRACTION_DATE)
WHERE acc.PRODUCT_LINE='ACCOUNTS'
AND acc.EXTRACTION_DATE IN (
    '2025-06-30','2025-07-31','2025-08-31',
    '2025-09-30','2025-10-31','2025-11-30'
)
""")


In [ ]:
base_bal_df["MONTH_DIFF"] = (
    (pd.to_datetime(base_bal_df["SNAPSHOT_DATE"]).dt.year -
     pd.to_datetime(base_bal_df["Av_Bal_Date"]).dt.year) * 12
    +
    (pd.to_datetime(base_bal_df["SNAPSHOT_DATE"]).dt.month -
     pd.to_datetime(base_bal_df["Av_Bal_Date"]).dt.month)
)


In [ ]:


base_bal_df["SNAPSHOT_DATE"] = pd.to_datetime(base_bal_df["SNAPSHOT_DATE"])
base_bal_df["Av_Bal_Date"] = pd.to_datetime(base_bal_df["Av_Bal_Date"])


In [ ]:
base_bal_df["MONTH_DIFF"] = (
    (base_bal_df["SNAPSHOT_DATE"].dt.year - base_bal_df["Av_Bal_Date"].dt.year) * 12
    +
    (base_bal_df["SNAPSHOT_DATE"].dt.month - base_bal_df["Av_Bal_Date"].dt.month)
)


In [ ]:
monthly_pivot = (
    monthly_df
    .pivot_table(
        index=["CUSTOMER_ID", "SNAPSHOT_DATE"],
        columns="MONTH_DIFF",
        values="CUMULATIVE_BAL_LCY",
        fill_value=0
    )
)

monthly_pivot.columns = [f"M{int(col)+1}" for col in monthly_pivot.columns]
monthly_pivot = monthly_pivot.reset_index()


In [ ]:
for i in range(1, 13):
    col = f"M{i}"
    if col not in monthly_pivot.columns:
        monthly_pivot[col] = 0


In [ ]:
avg_bal_df = (
    monthly_pivot
    .copy()
)

avg_bal_df["AVG_BAL_12M"] = (
    avg_bal_df[[f"M{i}" for i in range(1,13)]].sum(axis=1) / 12
)

avg_bal_df = avg_bal_df[["CUSTOMER_ID", "SNAPSHOT_DATE", "AVG_BAL_12M"]]


In [ ]:
final_df = avg_bal_df.merge(
    monthly_pivot,
    on=["CUSTOMER_ID", "SNAPSHOT_DATE"],
    how="left"
)


In [ ]:
for i in range(1, 12):
    col1 = f"M{i}"
    col2 = f"M{i+1}"
    new_col = f"M{i}M{i+1}_Change"
    
    final_df[new_col] = (
        (final_df[col1] - final_df[col2]) /
        final_df[col2].replace(0, pd.NA)
    ).round(2)


In [ ]:
final_df["Q1Q2_Change"] = (
    (final_df["M1"] + final_df["M2"] + final_df["M3"] -
     final_df["M4"] - final_df["M5"] - final_df["M6"]) /
    (final_df["M4"] + final_df["M5"] + final_df["M6"]).replace(0, pd.NA)
).round(2)

final_df["Q2Q3_Change"] = (
    (final_df["M4"] + final_df["M5"] + final_df["M6"] -
     final_df["M7"] - final_df["M8"] - final_df["M9"]) /
    (final_df["M7"] + final_df["M8"] + final_df["M9"]).replace(0, pd.NA)
).round(2)

final_df["Q3Q4_Change"] = (
    (final_df["M7"] + final_df["M8"] + final_df["M9"] -
     final_df["M10"] - final_df["M11"] - final_df["M12"]) /
    (final_df["M10"] + final_df["M11"] + final_df["M12"]).replace(0, pd.NA)
).round(2)


In [ ]:
base_cols = ["CUSTOMER_ID", "SNAPSHOT_DATE"]
avg_col = ["AVG_BAL_12M"]
month_cols = [f"M{i}" for i in range(1,13)]
trend_cols = [col for col in final_df.columns if "Change" in col]

final_df = final_df[base_cols + avg_col + month_cols + trend_cols]


In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

final_df.head()


In [ ]:
ppc_df = run_query("""
WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30'
    )
),
                   
FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
),
LatestPPC AS (
    SELECT
        p.CUSTOMER,
        s.SNAPSHOT_DATE,
        MAX(p.LOAD_DATE) AS MAX_LOAD_DATE
    FROM dbcba.RETAIL_PPC p
    JOIN SnapshotDates s
      ON p.LOAD_DATE <= s.SNAPSHOT_DATE
    AND p.CUSTOMER IN (SELECT CUSTOMER FROM FilteredCustomers)
    GROUP BY p.CUSTOMER, s.SNAPSHOT_DATE
)
SELECT
    s.SNAPSHOT_DATE,
    l.CUSTOMER AS CUSTOMER_ID,
    r.PPC
FROM SnapshotDates s
LEFT JOIN LatestPPC l
       ON s.SNAPSHOT_DATE = l.SNAPSHOT_DATE
LEFT JOIN dbcba.RETAIL_PPC r
       ON r.CUSTOMER = l.CUSTOMER
      AND r.LOAD_DATE = l.MAX_LOAD_DATE
ORDER BY s.SNAPSHOT_DATE, l.CUSTOMER;
""")
ppc_df.head()


In [ ]:
cc_df = run_query("""WITH SnapshotDates AS (
    SELECT DISTINCT EXTRACTION_DATE AS SNAPSHOT_DATE
    FROM dbcba.KE_Accounts_Master
    WHERE EXTRACTION_DATE IN (
        '2025-06-30','2025-07-31','2025-08-31',
        '2025-09-30','2025-10-31','2025-11-30'
    )
)
FilteredCustomers AS (
    SELECT am.CUSTOMER
    FROM dbcba.KE_Accounts_List am
    LEFT JOIN dbcba.KE_Customer_Master cm
        ON cm.CUSTOMER_NUMBER = am.CUSTOMER
    WHERE am.PRODUCT_LINE='LENDING'
      AND am.ARR_STATUS='CURRENT'
      AND cm.BUSINESS_SEGMENT IN ('270','250')
      AND COALESCE(am.ONLINE_ACTUAL_BAL,0) < 0
      AND am.CUSTOMER IS NOT NULL
),
SELECT
    s.SNAPSHOT_DATE,
    al.CUSTOMER AS CUSTOMER_ID,
    case
	    when vas.OVERDUEDAYS >0 then vas.OVERDUEDAYS
	    else 0
	    end as cc_OVERDUEDAYS,
    1 AS HAS_CREDIT_CARD
FROM SnapshotDates s
LEFT JOIN stgke.STG_OVERDUE_VAS vas
       ON vas.EXTRACTION_DATE = s.SNAPSHOT_DATE
      AND vas.PRODUCT_NAME NOT IN (
            'TZ USD PERSONAL credit GOLD',
            'TZS Personal Gold credit card',
            'Uganda Shillings Personal Gold Credit Card',
            'Uganda USD Visa Gold Credit',
            'Uganda business Credit',
            'TZ USD Business Gold credit',
            'TZS Business Gold Credit Card',
            'TZ Person Classic credit card',
            'Uganda Visa Classic Credit',
            'Uganda USD Business Credit',
            'TZ USD BUSSINESS credit SILVER'
      )
LEFT JOIN dbcba.KE_Accounts_List al
       ON vas.BANKACCOUNT = al.ACCOUNT_NUMBER
WHERE al.CUSTOMER IN (SELECT CUSTOMER FROM FilteredCustomers)
ORDER BY s.SNAPSHOT_DATE, al.CUSTOMER;
""")
cc_df.head()

In [ ]:
df_final = base_df \
    .merge(dto_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left') \
    .merge(cto_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left') \
    .merge(FID_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left') \
    .merge(maxdpd_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left') \
    .merge(cc_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left') \
     .merge(unpaid_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left') \
    .merge(ppc_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left') \
    .merge(avg_bal_df, on=['CUSTOMER_ID','SNAPSHOT_DATE'], how='left')
df_final.head()

In [ ]:
#Save final df to Parquet
df_final.to_parquet('final_feature_set.parquet', index=False)

In [ ]:
#Read parquet back to confirm
final_check_df = pd.read_parquet('final_feature_set.parquet')
final_check_df.head()

In [ ]:
# Create a cursor
cursor = conn.cursor()

In [ ]:
dataframes = {}

from tqdm import tqdm

for i, query in enumerate(tqdm(sql_queries, desc="Executing queries")):
    cursor.execute(query)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
    dataframes[f"df_{i+1}"] = df

In [ ]:
for key, df in dataframes.items():
    print(f"DataFrame {key}:")
    print(df.shape)
    print()

In [ ]:
#closing the connection
cursor.close()
conn.close()

In [ ]:
df_list = [df for df in dataframes.values()]
#the data frames
ctos_trended = df_list[0] #CUSTOMER
dtos_trended = df_list[1] #CUSTOMER
unpaid_items_trended = df_list[2] #CUSTOMER
cust_static_data = df_list[3] #CUSTOMER_NUMBER
max_dpds = df_list[4] #CUSTOMER
loan_to_income_ratio = df_list[5] #CUSTOMER_ID
avg_bal_trended = df_list[6] #CUSTOMER
fids = df_list[7] #customer_number
train_test__crb_raw = df_list[8] #CUSTOMER

In [ ]:
break

In [ ]:
fids.head()

In [ ]:
train_test__crb_raw['CUSTOMER'].nunique()

In [ ]:
loan_to_income_ratio.sort_values(by=['TOTAL_LOANS'], ascending=False).tail()

In [ ]:
#Function to merge all the dataframes
def merge_dataframes(dfs, merge_cols):
    """
    Merges multiple DataFrames on their respective merge columns, ensuring all records from the first df are maintained.
    Handles missing values in merge columns by dropping them and storing them separately.
    
    Parameters:
    dfs (list of pd.DataFrame): List of DataFrames to merge.
    merge_cols (list of str): List of column names used for merging, corresponding to each df.
    
    Returns:
    tuple: (Merged DataFrame, DataFrame of dropped rows with their source DataFrame name)
    """
    if not dfs or len(dfs) != len(merge_cols):
        raise ValueError("Number of DataFrames and merge column names must be the same and non-empty")

    dropped_rows = []
    processed_dfs = []

    # Convert all column names to uppercase for consistency
    for i in range(len(dfs)):
        dfs[i].columns = dfs[i].columns.str.upper()
        merge_cols[i] = merge_cols[i].upper()
    
    # Convert all merge columns to string type and handle missing values
    for i, (df, col) in enumerate(zip(dfs, merge_cols)):
        if col not in df.columns:
            print(f"DataFrame {i} available columns: {df.columns.tolist()}")
            raise KeyError(f"Merge column '{col}' not found in DataFrame {i}.")
        
        df = df.copy()
        df[col] = df[col].astype(str).str.strip()
        
        # Identify and store dropped rows
        missing_rows = df[df[col].isna() | (df[col] == "")].copy()
        if not missing_rows.empty:
            missing_rows['source_df'] = f'DataFrame_{i}'
            dropped_rows.append(missing_rows)
        
        # Drop rows where merge column is missing
        df = df.dropna(subset=[col])
        df = df[df[col] != ""]
        processed_dfs.append(df)

    # Store dropped rows in a separate DataFrame
    dropped_df = pd.concat(dropped_rows, ignore_index=True) if dropped_rows else pd.DataFrame()
    
    # Use the first DataFrame as the base
    merged_df = processed_dfs[0].copy()
    base_merge_col = merge_cols[0]
    
    # Merge each additional DataFrame
    for i in range(1, len(processed_dfs)):
        df, col = processed_dfs[i], merge_cols[i]
        print(f"Merging with DataFrame {i} on column '{col}'")  # Debug message
        
        if col not in df.columns:
            print(f"DataFrame {i} available columns: {df.columns.tolist()}")
            raise KeyError(f"Merge column '{col}' not found in DataFrame {i}.")
        
        merged_df = merged_df.merge(df, left_on=base_merge_col, right_on=col, how='left')
        
        # Drop the redundant merge column from the joined DataFrame only if it's not the base column
        if col != base_merge_col and col in merged_df.columns:
            merged_df.drop(columns=[col], inplace=True)
    
    return merged_df, dropped_df



In [ ]:
unpaid_items_trended.head(2)

In [ ]:
#Apply
dfs = [train_test__crb_raw, ctos_trended, dtos_trended, unpaid_items_trended, cust_static_data, max_dpds, loan_to_income_ratio, avg_bal_trended, fids]
merge_cols = ['CUSTOMER', 'CUSTOMER', 'CUSTOMER','CUSTOMER', 'CUSTOMER_NUMBER', 'CUSTOMER', 'CUSTOMER_ID', 'CUSTOMER', 'customer_number']
merged_df, dropped_df = merge_dataframes(dfs, merge_cols)

In [ ]:
merged_df.shape

In [ ]:
merged_df.head(2)

In [ ]:
dropped_df.shape

In [ ]:
dropped_df.head()

In [ ]:
#Use this for columns to drop
potential_drop_cols = ['AVG_INCOME', 'TOTAL_LOANS', 'MAX_OD_DAYS', 'MAX_LOAN_DPD']

In [ ]:
merged_df.head(1)

In [ ]:
#Create a target column
#merged_df['TARGET'] = merged_df['MAX_DPD_TRAIN_TEST'].apply(lambda x: 1 if x >0 and x<=5 else 0)
merged_df['TARGET'] = merged_df['MAX_DPD_TRAIN_TEST'].apply(lambda x: 1 if pd.notna(x) and 0 < x <= 5 else 0)

In [ ]:
merged_df['TARGET'].value_counts()

In [ ]:
merged_df.sort_values(by=['MAX_DPD_TRAIN_TEST'], ascending=False).tail()

In [ ]:
merged_df.dtypes

EDA!!

In [ ]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(merged_df).head(27)

In [ ]:
# missingness visually
missing_values_per_variable = missing_values_table(merged_df).rename_axis('Variables').reset_index() 

# plt.figure(figsize = (18, 8))
sns.barplot(data = missing_values_per_variable, x = 'Variables', y = '% of Total Values')
plt.xticks(rotation = 90)
plt.title("Proportion of Missing Values per Variable")
plt.ylabel("Percentage of Missing Values")
plt.show()

In [ ]:
features = [col for col in merged_df.columns.values if col not in ['CUSTOMER', 'MAX_DPD_TRAIN_TEST']]

merged_df[features].describe().T

In [ ]:
#Distribution of the features

#merged_df[features].hist(figsize = (15, 30), layout = (10, 3)); 
import math
import matplotlib.pyplot as plt

# Define number of features
num_features = len(features)

# Calculate optimal rows and columns
cols = 5  # You can adjust this
rows = math.ceil(num_features / cols)

# Plot histograms
merged_df[features].hist(figsize=(15, rows * 3), layout=(rows, cols))
plt.tight_layout()
plt.show()


In [ ]:
merged_df.columns

In [ ]:
features_2 = merged_df[['TU_NUMBEROF_ACCOUNTS', 'TU_OPENACCOUNT',
       'TU_PERFORMING', 'TU_NONPERFORMING', 'TU_CLOSED', 'TU_CURRENTINARREARS',
       'TU_ARREARS0DAYS', 'TU_MAXARREARS', 'TU_MAXARREARSLAST12MONTHS',
       'TU_MAXARREARSLAST3MONTHS', 'TU_PRINCIPALAMOUNT', 'TU_SCHEDULEDPAYMENTAMT',
       'TU_CREDITAPPLICATIONSDONE', 'TU_CURRENTBALANCEAMOUNT',
       'TU_SUBSCRIBERCURRENTBALANCEAMOUNT', 'TU_PASTDUEAMOUNT', 'TU_NUMBEROFENQUIRIES',
       'TU_ENQUIRIES30DAYS', 'TU_ENQUIRIES90DAYS', 'CRB_TU_SCORE', 'TU_MOBILE_TOTAL',
       'TU_MOBILE_PERFORMING', 'TU_MOBILE_NONPERFORMING', 'TU_MOBILE_PRINCIPALAMOUNT',
       'M1M2_CTO_AMT_CHANGE_PCT', 'M2M3_CTO_AMT_CHANGE_PCT',
       'M3M4_CTO_AMT_CHANGE_PCT', 'M4M5_CTO_AMT_CHANGE_PCT',
       'M5M6_CTO_AMT_CHANGE_PCT', 'M6M7_CTO_AMT_CHANGE_PCT',
       'M7M8_CTO_AMT_CHANGE_PCT', 'M8M9_CTO_AMT_CHANGE_PCT',
       'M9M10_CTO_AMT_CHANGE_PCT', 'M10M11_CTO_AMT_CHANGE_PCT',
       'M11M12_CTO_AMT_CHANGE_PCT', 'Q1Q2_CTO_AMT_CHANGE_PCT',
       'Q2Q3_CTO_AMT_CHANGE_PCT', 'Q3Q4_CTO_AMT_CHANGE_PCT',
       'Q1Q2_CTO_CNT_CHANGE_PCT', 'Q2Q3_CTO_CNT_CHANGE_PCT',
       'Q3Q4_CTO_CNT_CHANGE_PCT', 'M1M2_DTO_AMT_CHANGE_PCT',
       'M2M3_DTO_AMT_CHANGE_PCT', 'M3M4_DTO_AMT_CHANGE_PCT',
       'M4M5_DTO_AMT_CHANGE_PCT', 'M5M6_DTO_AMT_CHANGE_PCT',
       'M6M7_DTO_AMT_CHANGE_PCT', 'M7M8_DTO_AMT_CHANGE_PCT',
       'M8M9_DTO_AMT_CHANGE_PCT', 'M9M10_DTO_AMT_CHANGE_PCT',
       'M10M11_DTO_AMT_CHANGE_PCT', 'M11M12_DTO_AMT_CHANGE_PCT',
       'Q1Q2_DTO_AMT_CHANGE_PCT', 'Q2Q3_DTO_AMT_CHANGE_PCT',
       'Q3Q4_DTO_AMT_CHANGE_PCT', 'Q1Q2_DTO_CNT_CHANGE_PCT',
       'Q2Q3_DTO_CNT_CHANGE_PCT', 'Q3Q4_DTO_CNT_CHANGE_PCT', 'QI_UNPAID_VALUE',
       'Q2_UNPAID_VALUE', 'Q3_UNPAID_VALUE', 'Q4_UNPAID_VALUE',
       'Q1_UNPAID_COUNTS', 'Q2_UNPAID_COUNTS', 'Q3_UNPAID_COUNTS',
       'Q4_UNPAID_COUNTS', 'Q1Q2_UNPAID_AMT_CHANGE_PCT',
       'Q2Q3_UNPAID_AMT_CHANGE_PCT', 'Q3Q4_UNPAID_AMT_CHANGE_PCT',
       'Q1Q2_UNPAID_CNT_CHANGE_PCT', 'Q2Q3_UNPAID_CNT_CHANGE_PCT',
       'Q3Q4_UNPAID_CNT_CHANGE_PCT',
       'CURRMONTHPREVMONTH_UNPAID_AMT_CHANGE_PCT',
       'CURRMONTHPREVMONTH_UNPAID_CNT_CHANGE_PCT', 'AGE', 'CUST_TENURE_MONTHS',
       'M8M9_BAL_AMT_CHANGE_PCT', 'M9M10_BAL_AMT_CHANGE_PCT',
       'M10M11_BAL_AMT_CHANGE_PCT', 'M11M12_BAL_AMT_CHANGE_PCT',
       'Q1Q2_BAL_AMT_CHANGE_PCT', 'Q2Q3_BAL_AMT_CHANGE_PCT',
       'Q3Q4_BAL_AMT_CHANGE_PCT', 'NO_OF_ACCOUNTS_FID']]

In [ ]:
# Histograms scaled- an alternative better view to the above
%matplotlib inline
#axes = features_2.hist(figsize=(15, 30), layout=(59, 3), bins=20)
'''
# Apply log scale to y-axis for each subplot
for ax in axes.flatten():
    ax.set_yscale('log')  

# Display the plot
plt.tight_layout()  # Adjust layout to prevent overlap
#plt.show()'
'''

Kurtosis and Skewness

In [ ]:
#merged_df.columns

In [ ]:
coi = ['TU_NUMBEROF_ACCOUNTS', 'TU_OPENACCOUNT',
       'TU_PERFORMING', 'TU_NONPERFORMING', 'TU_CLOSED', 'TU_CURRENTINARREARS',
       'TU_ARREARS0DAYS', 'TU_MAXARREARS', 'TU_MAXARREARSLAST12MONTHS',
       'TU_MAXARREARSLAST3MONTHS', 'TU_PRINCIPALAMOUNT', 'TU_SCHEDULEDPAYMENTAMT',
       'TU_CREDITAPPLICATIONSDONE', 'TU_CURRENTBALANCEAMOUNT',
       'TU_SUBSCRIBERCURRENTBALANCEAMOUNT', 'TU_PASTDUEAMOUNT', 'TU_NUMBEROFENQUIRIES',
       'TU_ENQUIRIES30DAYS', 'TU_ENQUIRIES90DAYS', 'CRB_TU_SCORE', 'TU_MOBILE_TOTAL',
       'TU_MOBILE_PERFORMING', 'TU_MOBILE_NONPERFORMING', 'TU_MOBILE_PRINCIPALAMOUNT',
       'M1M2_CTO_AMT_CHANGE_PCT', 'M2M3_CTO_AMT_CHANGE_PCT',
       'M3M4_CTO_AMT_CHANGE_PCT', 'M4M5_CTO_AMT_CHANGE_PCT',
       'M5M6_CTO_AMT_CHANGE_PCT', 'M6M7_CTO_AMT_CHANGE_PCT',
       'M7M8_CTO_AMT_CHANGE_PCT', 'M8M9_CTO_AMT_CHANGE_PCT',
       'M9M10_CTO_AMT_CHANGE_PCT', 'M10M11_CTO_AMT_CHANGE_PCT',
       'M11M12_CTO_AMT_CHANGE_PCT', 'Q1Q2_CTO_AMT_CHANGE_PCT',
       'Q2Q3_CTO_AMT_CHANGE_PCT', 'Q3Q4_CTO_AMT_CHANGE_PCT',
       'Q1Q2_CTO_CNT_CHANGE_PCT', 'Q2Q3_CTO_CNT_CHANGE_PCT',
       'Q3Q4_CTO_CNT_CHANGE_PCT', 'M1M2_DTO_AMT_CHANGE_PCT',
       'M2M3_DTO_AMT_CHANGE_PCT', 'M3M4_DTO_AMT_CHANGE_PCT',
       'M4M5_DTO_AMT_CHANGE_PCT', 'M5M6_DTO_AMT_CHANGE_PCT',
       'M6M7_DTO_AMT_CHANGE_PCT', 'M7M8_DTO_AMT_CHANGE_PCT',
       'M8M9_DTO_AMT_CHANGE_PCT', 'M9M10_DTO_AMT_CHANGE_PCT',
       'M10M11_DTO_AMT_CHANGE_PCT', 'M11M12_DTO_AMT_CHANGE_PCT',
       'Q1Q2_DTO_AMT_CHANGE_PCT', 'Q2Q3_DTO_AMT_CHANGE_PCT',
       'Q3Q4_DTO_AMT_CHANGE_PCT', 'Q1Q2_DTO_CNT_CHANGE_PCT',
       'Q2Q3_DTO_CNT_CHANGE_PCT', 'Q3Q4_DTO_CNT_CHANGE_PCT', 'QI_UNPAID_VALUE',
       'Q2_UNPAID_VALUE', 'Q3_UNPAID_VALUE', 'Q4_UNPAID_VALUE',
       'Q1_UNPAID_COUNTS', 'Q2_UNPAID_COUNTS', 'Q3_UNPAID_COUNTS',
       'Q4_UNPAID_COUNTS', 'Q1Q2_UNPAID_AMT_CHANGE_PCT',
       'Q2Q3_UNPAID_AMT_CHANGE_PCT', 'Q3Q4_UNPAID_AMT_CHANGE_PCT',
       'Q1Q2_UNPAID_CNT_CHANGE_PCT', 'Q2Q3_UNPAID_CNT_CHANGE_PCT',
       'Q3Q4_UNPAID_CNT_CHANGE_PCT',
       'CURRMONTHPREVMONTH_UNPAID_AMT_CHANGE_PCT',
       'CURRMONTHPREVMONTH_UNPAID_CNT_CHANGE_PCT', 'AGE', 'CUST_TENURE_MONTHS',
       'M8M9_BAL_AMT_CHANGE_PCT', 'M9M10_BAL_AMT_CHANGE_PCT',
       'M10M11_BAL_AMT_CHANGE_PCT', 'M11M12_BAL_AMT_CHANGE_PCT',
       'Q1Q2_BAL_AMT_CHANGE_PCT', 'Q2Q3_BAL_AMT_CHANGE_PCT',
       'Q3Q4_BAL_AMT_CHANGE_PCT', 'NO_OF_ACCOUNTS_FID']

In [ ]:
skew = {}
kurt = {}

for i in coi:
    merged_df[i] = pd.to_numeric(merged_df[i], errors='coerce')  # Converts to float, sets invalid values to NaN
    skew[i] = merged_df[i].skew()
    kurt[i] = merged_df[i].kurt()

In [ ]:
# convert the dict to df
skewness_df = pd.DataFrame(list(skew.items()), columns=['Variable', 'Skewness'])
kurtosis_df = pd.DataFrame(list(kurt.items()), columns=['Variable', 'Kurtosis'])

skewness_kurtosis_df = pd.merge(skewness_df, kurtosis_df, on='Variable')
skewness_kurtosis_df

In [ ]:
plt.figure(figsize = (15, 4))

plt.plot(list(skew.keys()), list(skew.values()))
plt.xticks(rotation = 90, horizontalalignment = 'left')
plt.xlabel("Value for Skewness")
plt.title("Skewness for potential variables")
plt.show()

### **Interpreting Skewness and Kurtosis:**
Skewness and kurtosis are statistical measures that describe the shape of a distribution.

#### **1. Skewness Interpretation**
- **Skewness = 0** → Symmetric distribution
- **Skewness > 0** → Right-skewed (longer tail on the right)
- **Skewness < 0** → Left-skewed (longer tail on the left)
- **High Skewness (> 2 or < -2)** → Strongly asymmetric distribution

#### **2. Kurtosis Interpretation**
- **Kurtosis = 3** → Normal distribution (Mesokurtic)
- **Kurtosis > 3** → Heavy-tailed distribution (Leptokurtic, extreme outliers)
- **Kurtosis < 3** → Light-tailed distribution (Platykurtic, fewer extreme values)

---

### **Key Observations**
#### **1. Extremely High Skewness and Kurtosis**
- Many variables (e.g., `M1M2_CTO_AMT_CHANGE_PCT`, `M4M5_CTO_AMT_CHANGE_PCT`, `Q3Q4_DTO_AMT_CHANGE_PCT`) have **very high skewness (> 200)** and **extreme kurtosis (> 100,000)**.
- **Implication:** These variables have highly skewed distributions with extreme outliers.

#### **2. Negative Skewness**
- `Q1Q2_UNPAID_AMT_CHANGE_PCT` (-98.61), `Q3Q4_UNPAID_CNT_CHANGE_PCT` (-110.17), and similar variables are negatively skewed.
- **Implication:** The majority of values are on the higher end, with a long tail on the left.

#### **3. Customer Variable**
- `CUSTOMER`: Skewness (-0.29) & Kurtosis (-0.85) are close to 0.
- **Implication:** This variable is roughly symmetric and does not have extreme tails.

---

### **Implication**
1. **Severe Outliers Exist**  
   - The high kurtosis values indicate the presence of extreme values.
   - Needs to do **outlier treatment** (e.g., winsorization, log transformation).  
   - Consider robust methods like decision trees or quantile regression

In [ ]:
merged_df.head(2)

In [ ]:
merged_df.head(1)

In [ ]:
break

In [ ]:
merged_df.head(2)

Correlation

In [ ]:
plt.figure(figsize=(30, 15))
# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(merged_df[coi].corr())

# using the upper triangle matrix as mask 
sns.heatmap(merged_df[coi].corr(), annot=True, mask=matrix, cmap = "coolwarm")

In [ ]:
#Get closely corr cols into a table

def extract_high_correlations(df, coi, threshold=0.8):
    """
    Extracts pairs of columns with correlation above a given threshold.

    Parameters:
    - df: DataFrame containing the data
    - coi: List of columns to check for correlations
    - threshold: Minimum correlation value to consider (default is 0.8)

    Returns:
    - DataFrame with columns: Variable_1, Variable_2, Correlation
    """
    corr_matrix = df[coi].corr()
    upper_triangle = np.triu(np.ones(corr_matrix.shape), k=1)  # Mask to exclude duplicates and self-correlation
    
    correlated_pairs = []
    
    for i in range(len(coi)):
        for j in range(i + 1, len(coi)):  # Only consider upper triangle to avoid duplicates
            if abs(corr_matrix.iloc[i, j]) >= threshold:
                correlated_pairs.append({
                    "Variable_1": coi[i],
                    "Variable_2": coi[j],
                    "Correlation": corr_matrix.iloc[i, j]
                })
    
    return pd.DataFrame(correlated_pairs)

# Apply
correlated_vars = extract_high_correlations(merged_df, coi, threshold=0.8)

In [ ]:
correlated_vars.shape

In [ ]:
correlated_vars.sort_values(by=['Correlation'], ascending=False).head(20)

For unpaid items, QoQ changes for amounts are highly correlated to that of amounts, so can drop the count changes

In [ ]:
#Pick cols from above to exclude based on business knowledge
related_cols_list = [
    'TU_MOBILE_PRINCIPALAMOUNT',
    'TU_ARREARS0DAYS',
    'TU_MOBILE_PERFORMING',
    'TU_NONPERFORMING',
    'TU_CLOSED',
    'TU_SCHEDULEDPAYMENTAMT',
    'TU_MOBILE_PRINCIPALAMOUNT',
    
    'Q2_UNPAID_VALUE',
    'Q3_UNPAID_VALUE',
    'Q4_UNPAID_VALUE',
    'Q1_UNPAID_COUNTS',
    'Q2_UNPAID_COUNTS',
    'Q3_UNPAID_COUNTS',
    'Q4_UNPAID_COUNTS'
 
    ] #'Q1_UNPAID_VALUE',

merged_df_1 = merged_df.drop(columns=related_cols_list)

In [ ]:
merged_df_1.head()

In [ ]:
#send data to csv for prod_pipeline set- before sign_transform
today = datetime.today().strftime('%Y-%m-%d')
path = r'C:/Users/dennis.oseki/OneDrive - NCBA Group PLC/Retail/Early_Warning/prediction_data'
merged_df_1.to_csv(path + '/raw_data_1.csv', index=False)

In [ ]:
break

Transform before model training

In [ ]:
#Use this to transform while keeping the sign which is important in this case
#Transforms the data into a more normal-like distribution while preserving the sign.
#Handles both positive and negative values, unlike a standard log transform which fails for negatives.
#Reduces the effect of extreme values (outliers).

def signed_log_transform(feature):
    return np.sign(feature) * np.log1p(abs(feature))

'''   
df_transformed = df.copy()
for col in df.columns:
    df_transformed[col] = signed_log_transform(df[col])
'''   

In [ ]:
# Identify feature columns
feature_cols = [col for col in merged_df.columns if col not in ['CUSTOMER', 'TARGET', 'MAX_DPD_TRAIN_TEST']]

# Check for non-numeric columns
non_numeric_cols = merged_df[feature_cols].select_dtypes(exclude=['number']).columns
print("Non-numeric feature columns:", non_numeric_cols.tolist())

# Convert feature columns to numeric, coercing errors (non-convertible values will become NaN)
merged_df[feature_cols] = merged_df[feature_cols].apply(pd.to_numeric, errors='coerce')

# Apply the transformation only to feature columns
merged_df[feature_cols] = merged_df[feature_cols].applymap(signed_log_transform)

In [ ]:
break

In [ ]:
merged_df_1.shape

Model plan:

In [ ]:
# Store
customer_ids = merged_df_1['CUSTOMER']

# Define features and target
X = merged_df_1.drop(columns=['CUSTOMER', 'TARGET', 'MAX_DPD_TRAIN_TEST', 'MAX_OD_DAYS', 'MAX_LOAN_DPD'])
y = merged_df_1['TARGET']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=37)

# Split customer IDs so they match X_test
customer_ids_test = customer_ids.iloc[X_test.index]  # Get matching IDs for test set


In [ ]:
X_test.shape

In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [ ]:
X_test.head()

In [ ]:
#1. Train RandomForest as the base model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)
base_preds = base_model.predict_proba(X_train)[:,1]  # Get probabilities

In [ ]:
print(base_preds[:5])

In [ ]:
#2. Train XGBoost model as the first improvement from base
import xgboost as xgb

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train, sample_weight=abs(y_train - base_preds))  # Weight by errors
xgb_preds = xgb_model.predict_proba(X_train)[:,1]


In [ ]:
print(xgb_preds[:5])

In [ ]:
#3. Train final meta_learner model (LightGBM)
import lightgbm as lgb
import numpy as np

# Stack predictions as new features
X_meta = np.column_stack((base_preds, xgb_preds))

# Train LightGBM on the stacked predictions
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_meta, y_train)


import joblib
# Save the model to a file
model_path = r'C:/Users/dennis.oseki/OneDrive - NCBA Group PLC/Retail/Early_Warning/saved_model_versions'
date_time = datetime.now().strftime("%Y%m%d_%H%M%S")

#joblib.dump(lgb_model, f"{model_path}/base_model_{date_time}.pkl")
#joblib.dump(lgb_model, "meta_model.pkl")

In [ ]:
#Store xgb model
#joblib.dump(xgb_model, f"{model_path}/xgb_model_{date_time}.pkl")
#joblib.dump(lgb_model, "meta_model.pkl")

In [ ]:
#4. Generate final test set predictions using the ensemble
base_test_preds = base_model.predict_proba(X_test)[:,1]
xgb_test_preds = xgb_model.predict_proba(X_test)[:,1]

X_test_meta = np.column_stack((base_test_preds, xgb_test_preds))
final_preds = lgb_model.predict_proba(X_test_meta)

In [ ]:
print(final_preds[:7])

Model evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, log_loss, confusion_matrix,
    roc_curve, precision_recall_curve
)

def evaluate_model(y_true, y_pred, y_proba, model_name="Model Evaluation"):
    """
    Evaluates a classification model using key metrics and visualizations.

    Parameters:
    - y_true: Actual labels
    - y_pred: Predicted labels
    - y_proba: Predicted probabilities for class 1
    - model_name: Title for the visualizations
    """
    
    # Compute key metrics
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_proba)
    logloss = log_loss(y_true, y_proba)

    # Print results
    print(f"\n🔹 {model_name} Metrics")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"Log Loss: {logloss:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 5))

    # Confusion Matrix Plot
    plt.subplot(1, 3, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"{model_name} - Confusion Matrix")

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    plt.subplot(1, 3, 2)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}", color="blue")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{model_name} - ROC Curve")
    plt.legend()

    # Precision-Recall Curve
    precision_vals, recall_vals, _ = precision_recall_curve(y_true, y_proba)
    plt.subplot(1, 3, 3)
    plt.plot(recall_vals, precision_vals, label="Precision-Recall Curve", color="red")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"{model_name} - Precision-Recall Curve")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
#Apply 
# Get predictions from base model
y_pred = base_model.predict(X_test)  
y_proba = base_model.predict_proba(X_test)[:, 1]  # Probability of class 1

# Evaluate the model
evaluate_model(y_test, y_pred, y_proba, model_name="RF model Classifier")

In [ ]:
#Apply 
# Get predictions XGBoost- second model
y_pred = xgb_model.predict(X_test)  
y_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of class 1

# Evaluate the model
evaluate_model(y_test, y_pred, y_proba, model_name="XGBoost Classifier")

In [ ]:
#Apply to final model
# Get predictions from your trained XGBoost model
y_pred = lgb_model.predict(X_test_meta)  
y_proba = lgb_model.predict_proba(X_test_meta)[:, 1]  # Probability of class 1

# Evaluate the model
evaluate_model(y_test, y_pred, y_proba, model_name="Ensembled model Classifier")

In [ ]:
y_pred

In [ ]:
y_proba

In [ ]:
X_test.shape

In [ ]:
#Combining back to original data to get customer numbers to predictions of the ensembled model

test_set_predictions = pd.DataFrame({
    'CUSTOMER': customer_ids_test,  # Restore customer IDs
    'TARGET': y_test.values,  # Convert to avoid index mismatches
    'PREDICTIONS': y_pred,
    'PROBABILITIES': y_proba
})

In [ ]:
test_set_predictions.shape

In [ ]:
#test_set_predictions.sort_values(by=['PROBABILITIES'], ascending=False).tail()
test_set_predictions.head(15)

In [ ]:
#create global variables for reuse
results_path = r'C:/Users/dennis.oseki/OneDrive - NCBA Group PLC/Retail/Early_Warning/Test_Results'
date_time = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
#Send results to xlsx
test_set_predictions.to_excel(f"{results_path}/test_set_predictions_{date_time}.xlsx", index=False)

In [ ]:
#Combine back to original df to get all columns
X_test_original = merged_df.loc[X_test.index]  # Get original X_test rows with all columns

test_set_predictions_all = X_test_original.merge(test_set_predictions, on='CUSTOMER', how='left')

In [ ]:
test_set_predictions_all.head()

In [ ]:
test_set_predictions_all.shape

In [ ]:
#Feature importance
feature_importances = xgb_model.feature_importances_

# pair with the corresponding feature names for better interpretation
feature_names = X.columns 
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

print('Feature importance V1.0:')
print()
#Print
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.4f}")

In [ ]:
#Send feature importance to xlsx
feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": feature_importances.astype(float)
}).round(4)

# Sort by importance
feature_importance_sorted = feature_importance_df.sort_values(by="Importance", ascending=False).reset_index(drop=True)

In [ ]:
feature_importance_sorted.head()

In [ ]:
#Send feature importance to xlsx
#feature_importance_sorted.to_excel(f"{results_path}/feature_importance_{date_time}.xlsx", index=False)

In [ ]:
break

Alternative upstacking approach

In [ ]:
#pip install catboost

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

In [ ]:
# Define base models with hyperparameters
''' 
base_models = [
    ("rf", RandomForestClassifier(n_estimators=207, random_state=27)),
    ("xgb", XGBClassifier(n_estimators=207, learning_rate=0.15, random_state=27, use_label_encoder=False)),
    ("lgbm", LGBMClassifier(n_estimators=207, learning_rate=0.15, random_state=27)),
    ("cat", CatBoostClassifier(iterations=207, learning_rate=0.15, random_state=27, verbose=0))  # Set verbose=0 to suppress logs
]
'''

#Using GridSearch to tune hyperparams including class weight
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights based on training labels
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

# Define base models with class weights
base_models = {
    "rf": RandomForestClassifier(n_estimators=207, random_state=27, class_weight="balanced"),
    "xgb": XGBClassifier(n_estimators=207, learning_rate=0.15, random_state=27, use_label_encoder=False, 
                         scale_pos_weight=class_weight_dict[1]),  # Weighting the positive class
    "lgbm": LGBMClassifier(n_estimators=207, learning_rate=0.15, random_state=27, class_weight="balanced"),
   # "cat": CatBoostClassifier(iterations=207, learning_rate=0.15, random_state=27, verbose=0, scale_pos_weight=class_weight_dict[1])
}


# Define hyperparameter grids for each model
param_grids = {
    "rf": {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10]
    },
    "xgb": {
        "learning_rate": [0.05, 0.1, 0.15],
        "max_depth": [3, 6, 9],
        "subsample": [0.7, 0.8, 1.0]
    },
    "lgbm": {
        "learning_rate": [0.05, 0.1, 0.15],
        "max_depth": [-1, 5, 10],
        "num_leaves": [31, 50, 100]
    },
    "cat": {
        "learning_rate": [0.05, 0.1, 0.15],
        "depth": [4, 6, 8],
        "l2_leaf_reg": [3, 5, 7]
    }
}

# Define scoring metric (optimize for recall)
scoring_metric = make_scorer(recall_score)

# Perform Grid Search for each model
best_params = {}

for model_name, model in base_models.items():
    print(f"Running GridSearchCV for {model_name}...")
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring=scoring_metric, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)  # Make sure X_train and y_train are defined
    
    best_params[model_name] = grid_search.best_params_
    print(f"Best params for {model_name}: {grid_search.best_params_}")
    print(f"Best Recall Score: {grid_search.best_score_}\n")

# Print all best parameters
print("Final Best Hyperparameters for Each Model:")
print(best_params)

In [ ]:
# Create an empty array to store meta-features
'''
meta_features = np.zeros((X_train.shape[0], len(base_models)))

# Generate OOF predictions
for i, (name, model) in enumerate(base_models):
    meta_features[:, i] = cross_val_predict(
        model, X_train, y_train, cv=5, method="predict_proba"
    )[:, 1]  # Take probability of the positive class
'''

'''
from sklearn.model_selection import cross_val_predict
import numpy as np

# Create an empty array to store meta-features
meta_features = np.zeros((X_train.shape[0], len(base_models)))

# Use best hyperparameters from GridSearchCV results
optimized_models = {
    "rf": RandomForestClassifier(**best_params["rf"], random_state=27, class_weight="balanced"),
    "xgb": XGBClassifier(**best_params["xgb"], random_state=27, use_label_encoder=False,
                         scale_pos_weight=class_weight_dict[1]),
    "lgbm": LGBMClassifier(**best_params["lgbm"], random_state=27, class_weight="balanced"),
    #"cat": CatBoostClassifier(**best_params["cat"], random_state=27, verbose=0, scale_pos_weight=class_weight_dict[1])
}

# Generate Out-of-Fold (OOF) predictions
for i, (name, model) in enumerate(optimized_models.items()):
    print(f"Generating OOF predictions for {name}...")
    meta_features[:, i] = cross_val_predict(
        model, X_train, y_train, cv=5, method="predict_proba"
    )[:, 1]  # Take probability of the positive class

# Train meta-model on OOF predictions
meta_model = XGBClassifier(random_state=27) 
meta_model.fit(meta_features, y_train)

# Generate final predictions on test data
test_meta_features = np.zeros((X_test.shape[0], len(optimized_models)))
for i, (name, model) in enumerate(optimized_models.items()):
    model.fit(X_train, y_train)  # Fit each base model
    test_meta_features[:, i] = model.predict_proba(X_test)[:, 1]  # Take probability of positive class

# Final prediction using meta-model
final_preds = meta_model.predict(test_meta_features)

# Print performance metrics
from sklearn.metrics import classification_report
print(classification_report(y_test, final_preds))'
'''

In [ ]:
from sklearn.model_selection import cross_val_predict
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# Use best hyperparameters from GridSearchCV results
optimized_models = {
    "rf": RandomForestClassifier(**best_params["rf"], random_state=27, class_weight="balanced"),
    "xgb": XGBClassifier(**best_params["xgb"], random_state=27, use_label_encoder=False,
                         scale_pos_weight=class_weight_dict[1]),
    "lgbm": LGBMClassifier(**best_params["lgbm"], random_state=27, class_weight="balanced"),
    #"cat": CatBoostClassifier(**best_params["cat"], random_state=27, verbose=0, scale_pos_weight=class_weight_dict[1])
}

# Create an empty array to store OOF predictions
meta_features = np.zeros((X_train.shape[0], len(optimized_models)))

# Generate Out-of-Fold (OOF) predictions
for i, (name, model) in enumerate(optimized_models.items()):
    print(f"Generating OOF predictions for {name}...")
    meta_features[:, i] = cross_val_predict(
        model, X_train, y_train, cv=5, method="predict_proba"
    )[:, 1]  # Take probability of the positive class

# Concatenate original features + base model OOF predictions
meta_train = np.hstack([X_train, meta_features])

# Train meta-model
meta_model = XGBClassifier(random_state=27) 
meta_model.fit(meta_train, y_train)

# Generate test set base model predictions
test_meta_features = np.zeros((X_test.shape[0], len(optimized_models)))
for i, (name, model) in enumerate(optimized_models.items()):
    model.fit(X_train, y_train)  # Fit each base model
    test_meta_features[:, i] = model.predict_proba(X_test)[:, 1]  # Take probability of positive class

# Concatenate original test features + base model test predictions
meta_test = np.hstack([X_test, test_meta_features])

# Final prediction using meta-model
final_preds = meta_model.predict(meta_test)

# Print performance metrics
print(classification_report(y_test, final_preds))

In [ ]:
#meta_model = XGBClassifier(n_estimators=217, learning_rate=0.05, random_state=42,  scale_pos_weight=class_weight_dict[1])
#meta_model.fit(meta_features, y_train)

In [ ]:
#Test fine tuning further
'''
from sklearn.metrics import precision_recall_curve
import numpy as np

# Get predicted probabilities for the positive class
probs = meta_model.predict_proba(test_meta_features)[:, 1]

# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, probs)

# Choose a threshold that maximizes recall while keeping decent precision
desired_recall = 0.70  # Adjust based on trade-off preference
best_threshold_idx = np.argmax(recall >= desired_recall)
best_threshold = thresholds[best_threshold_idx]

# Apply new threshold
final_preds = (probs >= best_threshold).astype(int)
# Generate test set predictions for base models using optimized hyperparameters
meta_test = np.column_stack([
    optimized_models[name].fit(X_train, y_train).predict_proba(X_test)[:, 1] 
    for name in optimized_models
])

# Predict final outcome using meta-learner
final_predictions = meta_model.predict(meta_test)'
'''

In [ ]:
meta_features.shape

In [ ]:
print(meta_test.shape)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
print(len(optimized_models))

In [ ]:
#Apply to alternate final
# Get preds
#y_pred = lgb_model.predict(X_test_meta)  
#y_proba = lgb_model.predict_proba(X_test_meta)[:, 1]  # Probability of class 1

# Predict final outcome using meta-learner
y_pred = meta_model.predict(meta_test)           # Binary predictions
y_proba = meta_model.predict_proba(meta_test)[:, 1]  # Probabilities for the positive class

# Evaluate the model
evaluate_model(y_test, y_pred, y_proba, model_name="Alternate Ensemble Model")

In [ ]:
#Find best threshold for high recall
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Find the threshold where recall is at least 80% (or as high as possible)
recall_target = 0.80  # Adjust this if needed
best_idx = (recall >= recall_target).nonzero()[0][-1]  # Last index where recall is still >= 80%
best_threshold = thresholds[best_idx]

print(f"Best Threshold for High Recall: {best_threshold:.4f}")

# Apply the threshold
#input_data_2["predicted_class"] = (y_proba >= best_threshold).astype(int)


Early risk detection- should favor recall over precision (catch more defaulters)

In [ ]:
# Save the models
model_path = r'C:/Users/dennis.oseki/OneDrive - NCBA Group PLC/Retail/Early_Warning/saved_model_versions'
date_time = datetime.now().strftime("%Y%m%d_%H%M%S")

import joblib
import os
from datetime import datetime

# Define a directory to store models
model_path = r'C:/Users/dennis.oseki/OneDrive - NCBA Group PLC/Retail/Early_Warning/saved_model_versions'
os.makedirs(model_path, exist_ok=True)

# Timestamp for versioning
date_time = datetime.now().strftime("%Y%m%d%H%M%S")

# Save all base models
for name, model in optimized_models.items():
    joblib.dump(model, f"{model_path}/base_model_{name}_{date_time}.pkl")

# Save the meta-model
joblib.dump(meta_model, f"{model_path}/meta_model_{date_time}.pkl")

print(f"Models saved in {model_path}")

In [ ]:
#Save the transformer
import joblib
import numpy as np

# Custom Transformer Class
class SignedLogTransformer:
    def transform(self, feature):
        return np.sign(feature) * np.log1p(abs(feature))

# Instantiate transformer
log_transformer = SignedLogTransformer()

# Save the transformer
joblib.dump(log_transformer, f"{model_path}/log_transformer_{date_time}.pkl")
print("Transformer saved successfully!")

In [ ]:
# Base model prediction feature names
base_model_features = ["rf_pred", "xgb_pred", "lgbm_pred"]

#Feature importance
feature_importances_2 = meta_model.feature_importances_

# Combine original feature names with base model predictions
aligned_feature_names = list(X.columns) + base_model_features

# Check if the lengths now match
if len(feature_importances_2) == len(aligned_feature_names):
    # Create DataFrame
    feature_importance_df_meta = pd.DataFrame({
        "Feature": aligned_feature_names,
        "Importance": feature_importances_2.astype(float)
    }).round(6)

    # Sort by importance
    feature_importance_sorted_meta = feature_importance_df_meta.sort_values(
        by="Importance", ascending=False).reset_index(drop=True)

    # Save to Excel
    #feature_importance_sorted_meta.to_excel("feature_importance_meta.xlsx", index=False)
    #print("Feature importance saved successfully!")
else:
    print(f"Length mismatch: {len(feature_importances_2)} importances vs {len(aligned_feature_names)} features")


In [ ]:
feature_importance_sorted_meta.head(10)

In [ ]:
meta_model.n_features_in_

In [ ]:
len(feature_importances_2)

In [ ]:
len(feature_names)

In [ ]:
#Send feature importance to xlsx for the meta_model
feature_importance_sorted_meta.to_excel(f"{results_path}/feature_importance_meta_{date_time}.xlsx", index=False)

### Model Comparison: Alternate Ensemble vs. XGBoost

#### **1. Accuracy**
- **Ensemble:** 71.27%
- **XGBoost:** 69.02%
- **Better:** **Ensemble**

#### **2. Precision**
- **Ensemble:** 65.81%
- **XGBoost:** 61.54
- **Better:** **Ensemble** (fewer false positives)

#### **3. Recall**
- **Ensemble:** 57.8%
- **XGBoost:** 62.03%
- **Better:** **XGBoost** (fewer false negatives, better at catching positives)

#### **4. F1-score**
- **Ensemble:** 61.54%
- **XGBoost:** 61.78%
- **Better:** **XGBoost** (better balance between precision & recall)

#### **5. ROC AUC (Discrimination Power)**
- **Ensemble:** 0.7811
- **XGBoost:** 0.7668
- **Better:** **Ensemble** (better at ranking positive instances higher than negative ones)

#### **6. Log Loss (Lower is better)**
- **Ensemble:** 0.5329
- **XGBoost:** 0.5454
- **Better:** **Ensemble** (better probability calibration)

### **Overall Conclusion**
- **If you prioritize recall (catching more positive cases, reducing false negatives)** → **XGBoost is better.**
- **If you prioritize precision (reducing false positives) and overall ranking quality (AUC, log loss)** → **Ensemble is better.**
- **For a balanced trade-off, the ensemble appears to be slightly superior.** It has better precision, ROC AUC, log loss, and accuracy, although XGBoost has better recall and F1-score.